### Imports

In [9]:
import pandas as pd
import numpy as np
import pycountry

### Constants

In [10]:
# versions
EXIOBASE_VERSION = '3.8.2'
IGNITE_VERSION = '1.1'

# Rest of world regions as defined in Exiobase
ROW_AFRICA = 'Rest of World Africa'
ROW_AMERICA = 'Rest of World America'
ROW_ASIA = 'Rest of World Asia and Pacific'
ROW_EUROPE = 'Rest of World Europe'
ROW_ME = 'Rest of World Middle East'

# Exiobase column names

# Ignite column names

EF_CAT_L1 = 'EF Category L1'
EF_CAT_L2 = 'EF Category L2'
IPCC_AR5 = 'EF IPCC AR5 [kg CO2e/€]'

### Helper functions

In [11]:
def remove_numbers_from_region(region):
    return region[0:2]

def add_full_region_name(alpha_2):
    if alpha_2 == 'WF':
        return ROW_AFRICA
    elif alpha_2 == 'WL':
        return ROW_AMERICA
    elif alpha_2 == 'WA':
        return ROW_ASIA
    elif alpha_2 == 'WE':
        return ROW_EUROPE
    elif alpha_2 == 'WM':
        return ROW_ME
    else: 
        country = pycountry.countries.get(alpha_2=alpha_2)
        if country:
            return pycountry.countries.get(alpha_2=alpha_2).name
        else:
            return 'No country'

def scale_val(row, category_percentiles, global_upper_limit, global_lower_limit):
    sector = row[EF_CAT_L2]
    value = row[IPCC_AR5]
    local_lower_limit, local_upper_limit = category_percentiles.loc[category_percentiles[EF_CAT_L2] == sector][IPCC_AR5].values

    if value < 0 and value > -global_upper_limit:
        return [-value, True]

    if value < global_lower_limit and local_lower_limit < global_lower_limit:
        return [global_lower_limit, True]
    elif value > global_upper_limit and local_upper_limit > global_upper_limit:
        return [global_upper_limit, True]

    if value < local_lower_limit and not local_lower_limit == 0:
        return [local_lower_limit, True]
    elif value > local_upper_limit:
        return [local_upper_limit, True]
    elif value == 0 or value == np.nan:
        return [global_lower_limit, True]

    return [value, False]

### Load full raw database and create Ignite version

In [12]:
firstRun = True
temp_m = False

for year in range(2010, 2023):
    for data_type in ['pxp']:
        # not a very elegant solution, but it works
        if firstRun:
            firstRun = False
        else:
            temp_m = m_t

        SOURCE_FOLDER = 'raw-data/' + '/IOT_' + str(year) + '_' + data_type

        # load the Exiobase MRIO extension multipliers (M.txt) and gross/total output (x.txt) for the selected year and data_type
        m = pd.read_csv(SOURCE_FOLDER + '/impacts/M.txt', sep='\t', low_memory=False)
        x = pd.read_csv(SOURCE_FOLDER + '/x.txt', sep='\t')

        # select the relevant rows and transpose matrix
        m_t=m[m.region.isin(['sector', 'Water Consumption Blue - Total',
                        'GHG emissions AR5 (GWP100) | GWP100 (IPCC, 2010)', 
                        'Land use Crop, Forest, Pasture'])].transpose().reset_index()

        m_t.columns = ['region', 'sector','Water Consumption Blue - Total','GHG emissions AR5 (GWP100) | GWP100 (IPCC, 2010)', 'Land use Crop, Forest, Pasture']

        # remove first row which doesn't have any valuable information after transposing
        m_t = m_t.iloc[1:-1]

        # change from M€ to €
        m_t['EF IPCC AR5 [kg CO2e/€]']=(m_t['GHG emissions AR5 (GWP100) | GWP100 (IPCC, 2010)'].astype(float)/1000000).round(decimals = 8)

        # fix alpha2 naming
        m_t['EF Alpha2'] = m_t['region'].astype(str).apply(remove_numbers_from_region)

        # add full region/ROW name based on alpha 2 naming
        m_t['EF Region'] = m_t['EF Alpha2'].astype(str).apply(add_full_region_name)

        # rename and add relevant columns
        m_t.rename(columns={'Water Consumption Blue - Total':'Water Consumption [m3/€]', 'Land use Crop, Forest, Pasture':'Land Use [m2/€]'}, inplace=True)
        m_t['EF Year'] = str(year)
        m_t['EF Currency'] = 'EUR'
        m_t['EF Source'] = 'Exiobase ' + EXIOBASE_VERSION + ' (Ignite update ' + IGNITE_VERSION + ')'
        m_t['EF Category L1'] = 'Product'

        # clean up the naming convention of Exiobase
        m_t['EF Category L2'] = m_t['sector'].str.replace(r'\([^)]\d[^)]*\)', '', regex=True).str.strip()
        m_t['EF Category L2'] = m_t['EF Category L2'].str.replace('products of Vegetable oils and fats', 'Products of vegetable oils and fats')

        # set to False for all emission factors initially, then set to True if updated at a later stage
        m_t['Adjusted by Ignite'] = False

        # added to have a single text description of the unique combination of dimentions for the select emission factor
        m_t['EF Unique String'] = m_t['EF Category L1'].astype(str) + '-' + m_t['EF Category L2'].astype(str) + '-' + m_t['EF Region'].astype(str) + '-' + m_t['EF Year'].astype(str)
        
        # remove columns not longer needed
        m_t.drop(columns=['region', 'sector', 'GHG emissions AR5 (GWP100) | GWP100 (IPCC, 2010)'], inplace=True)

        # limit outliers 
        global_lower_limit, global_upper_limit = m_t['EF IPCC AR5 [kg CO2e/€]'].replace(0, np.nan).quantile([0.03,0.97]).values
        category_percentiles = m_t.replace(0, np.nan).groupby('EF Category L2')['EF IPCC AR5 [kg CO2e/€]'].quantile([0.05, 0.95]).reset_index()
        m_t[['Emission Factor [kg CO2e/€]', 'Adjusted by Ignite']] = m_t[['EF Category L2', 'EF IPCC AR5 [kg CO2e/€]']].apply(lambda row: scale_val(row, category_percentiles, global_upper_limit, global_lower_limit), axis=1, result_type='expand').values

        # add regional weighted averages
        x.rename(columns={'region':'EF Alpha2', 'sector':'EF Category L2'}, inplace=True)
        x['EF Category L2'] = x['EF Category L2'].str.replace(r'\([^)]\d[^)]*\)', '', regex=True).str.strip()
        x['EF Category L2'] = x['EF Category L2'].str.replace('products of Vegetable oils and fats', 'Products of vegetable oils and fats')
        x['sector_weights'] = x['indout'] / x.groupby('EF Alpha2')['indout'].transform('sum')

        number_columns = ['Water Consumption [m3/€]', 'Land Use [m2/€]', 'EF IPCC AR5 [kg CO2e/€]', 'Emission Factor [kg CO2e/€]']
        intermediate_df = m_t.merge(right=x, how='left', on=['EF Alpha2', 'EF Category L2'])
        intermediate_df[number_columns] = intermediate_df[number_columns].astype(float).multiply(intermediate_df['sector_weights'], axis='index', )
      
        region_averages = intermediate_df.groupby(['EF Region','EF Alpha2', 'EF Currency', 'EF Source', 'EF Year'])[number_columns].sum().reset_index()
        region_averages['EF Category L1'] = 'Regional average'
        region_averages['Adjusted by Ignite'] = True
        region_averages['EF Unique String'] = region_averages['EF Category L1'].astype(str) + '-' + region_averages['EF Region'].astype(str) + '-' + region_averages['EF Year'].astype(str)

        # set dtype of boolean column
        m_t['Adjusted by Ignite']=m_t['Adjusted by Ignite'].astype(bool)

        # add regional averages for the selected year
        m_t = pd.concat([m_t, region_averages], ignore_index=True)

        # add data to the already run years if not first year
        if type(temp_m) != bool:
            m_t = pd.concat([temp_m, m_t], ignore_index=True)

        print('Completed year:', year)

# reorder columns
m_t = m_t[['EF Category L1', 'EF Category L2', 'EF Region', 'EF Year', 'EF Currency', 'EF Source', 'EF Unique String', 'Adjusted by Ignite',  'EF IPCC AR5 [kg CO2e/€]', 'Water Consumption [m3/€]', 'Land Use [m2/€]', 'Emission Factor [kg CO2e/€]']]



/var/folders/bh/rcxt7tsx5rv2pq74snstw_fr0000gn/T/ipykernel_22067/3393507558.py:77: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  m_t = pd.concat([m_t, region_averages], ignore_index=True)


Completed year: 2010


/var/folders/bh/rcxt7tsx5rv2pq74snstw_fr0000gn/T/ipykernel_22067/3393507558.py:77: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  m_t = pd.concat([m_t, region_averages], ignore_index=True)


Completed year: 2011


/var/folders/bh/rcxt7tsx5rv2pq74snstw_fr0000gn/T/ipykernel_22067/3393507558.py:77: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  m_t = pd.concat([m_t, region_averages], ignore_index=True)


...

/var/folders/bh/rcxt7tsx5rv2pq74snstw_fr0000gn/T/ipykernel_22067/3393507558.py:77: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  m_t = pd.concat([m_t, region_averages], ignore_index=True)


Completed year: 2020


/var/folders/bh/rcxt7tsx5rv2pq74snstw_fr0000gn/T/ipykernel_22067/3393507558.py:77: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  m_t = pd.concat([m_t, region_averages], ignore_index=True)


Completed year: 2021
Completed year: 2022


/var/folders/bh/rcxt7tsx5rv2pq74snstw_fr0000gn/T/ipykernel_22067/3393507558.py:77: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  m_t = pd.concat([m_t, region_averages], ignore_index=True)


### Store the completed file (EUR-version) in the same directory

In [13]:
m_t.to_excel('EUR_Exiobase3_8_2-Ignite1_1-Products_2010-2022.xlsx', index=False)